In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
import torch
import datasets
import random

In [2]:
model_name = "rinna/japanese-gpt2-medium"

## トークナイザー
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    legacy = False
)

## LLM
model = AutoModelForCausalLM.from_pretrained(model_name)

## 演算にコンピュータのGPUを利用する
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32000, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=32000, bias=False)
)

In [26]:
"""
文章生成する
@param input_messages_series 一連の入力文章（文章の型はmdlに合わせる）
@param tkn トークナイザー
@param mdl モデル
"""
def generate_sentence(
    input_messages_series: str,
    tkn,
    mdl
):
    ##
    input_tokens = tkn.encode(
        input_messages_series,
        return_tensors = 'pt',
        add_special_tokens = False
    ).to(mdl.device)
    ##
    output_tokens = mdl.generate(
        input_tokens,
        # max_length = 200,
        # min_length = 200,
        max_new_tokens = 32,
        do_sample = True,
        top_k = 500,
        top_p = 0.95,
        pad_token_id = tkn.pad_token_id,
        bos_token_id = tkn.bos_token_id,
        eos_token_id = tkn.eos_token_id,
        bad_words_ids = [[tkn.unk_token_id]]
    )
    ##
    output_messages_series = tkn.decode(
        output_tokens[0],
        skip_special_tokens = True
    )
    
    ##
    return output_messages_series

In [27]:
%%time

## サンプル
## トレーニング前なので回答しないはず

print(generate_sentence(
    "小学校給食の無料化は何年をめどに達成されますか？",
    tokenizer,
    model
))

小学校給食の無料化は何年をめどに達成されますか? いままで400年間、無償で子どもたちにご飯を食べさせてきました。なぜ、無料にするのか? #nhk https://t
CPU times: user 464 ms, sys: 619 μs, total: 465 ms
Wall time: 463 ms


In [5]:
## データセットをロードする

ds = datasets.load_dataset("cl-nagoya/auto-wiki-qa")
ds

DatasetDict({
    train: Dataset({
        features: ['passage_id', 'query', 'answer', 'text', 'title', 'url'],
        num_rows: 2377503
    })
})

In [6]:
## 文字数が多すぎるサンプルは省く

## 文字数が多いものを含むと学習がうまくいかなかった
## 文字数が多いものに全トークン長を合わせると、パディングが多くなり、学習の質が低下する

ds = ds.filter(
    lambda example: len(example["text"]) < 100
)
ds

DatasetDict({
    train: Dataset({
        features: ['passage_id', 'query', 'answer', 'text', 'title', 'url'],
        num_rows: 132732
    })
})

In [7]:
ds["train"][110]

{'passage_id': 2832,
 'query': '6月4日は年始から何日目？',
 'answer': '155日目',
 'text': '6月4日(ろくがつよっか)は、グレゴリオ暦で年始から155日目(閏年では156日目)にあたり、年末まであと210日ある。',
 'title': '6月4日',
 'url': 'https://ja.wikipedia.org/wiki/6%E6%9C%884%E6%97%A5'}

## 対話の型を学習させる

cyberagentのLLMを参考にして、前情報 + 質問 + 回答のブロックを作成する。

[url](https://huggingface.co/cyberagent/calm3-22b-chat)

In [8]:
## 対話の型となるフレームワード
## cyberagentのLLMを参考にする
## https://huggingface.co/cyberagent/calm3-22b-chat

start_block = "<|im_start|>"
end_block = "<|im_end|>"

In [9]:
## ギリギリ全文を含めることができるトークン長
MAX_LENGTH = 128

## 一連の情報を一つの文章に変換する
## さらにトークナイズする
def serialize(example):
    sentence = ""
    sentence += start_block + "hint\n"
    sentence += example["text"] + end_block + "\n"
    sentence += start_block + "user\n"
    sentence += example["query"] + end_block + "\n"
    sentence += start_block + "assistant\n"
    sentence += example["answer"] + end_block + "\n"
    example["tmp"] = sentence
    return tokenizer(
        example["tmp"],
        max_length = MAX_LENGTH,
        padding = 'max_length',
        truncation = True
    )

ds = ds.map(
    serialize,
    remove_columns = ["query", "answer", "text", "title", "url"]    
)
ds

DatasetDict({
    train: Dataset({
        features: ['passage_id', 'tmp', 'input_ids', 'attention_mask'],
        num_rows: 132732
    })
})

In [19]:
## サンプルデータ
print(tokenizer.decode(ds["train"][10000]['input_ids']))

<|im_start|>hint ハンマーハウス(英:<unk>eam <unk>ammer <unk>ouse)は、アメリカ合衆国の総合格闘技のチーム。代表はマーク・コールマン。<|im_end|> <|im_start|>user ハンマーハウスの代表は誰?<|im_end|> <|im_start|>assistant マーク・コールマン<|im_end|></s>[PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]


In [11]:
ds = ds["train"].shuffle().train_test_split(test_size = 0.1)
ds

DatasetDict({
    train: Dataset({
        features: ['passage_id', 'tmp', 'input_ids', 'attention_mask'],
        num_rows: 119458
    })
    test: Dataset({
        features: ['passage_id', 'tmp', 'input_ids', 'attention_mask'],
        num_rows: 13274
    })
})

In [12]:
## チューニングパラメータ

num_train_epochs = 5
per_device_train_batch_size = 16
per_device_eval_batch_size = 16
learning_rate = 1e-5
weight_decay = 0.01

trained_model_name = 'seanayuuto/rinna-medium-dialogue'

In [14]:
## チューニングの準備

training_args = TrainingArguments(
    output_dir = f"../{trained_model_name}",
    eval_strategy = 'epoch',
    num_train_epochs = num_train_epochs,
    per_device_train_batch_size = per_device_train_batch_size,
    per_device_eval_batch_size = per_device_eval_batch_size,
    learning_rate = learning_rate,
    weight_decay = weight_decay,
    logging_steps = 100
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer = tokenizer,
    mlm = False
)

trainer = Trainer(
    model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = ds['train'].select(range(1600)),
    eval_dataset = ds['test'].select(range(400))
)

In [15]:
%%time
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,1.251600,0.919489
2,0.927700,0.872661
3,0.871600,0.853786
4,0.841500,0.845486
5,0.826200,0.842628


CPU times: user 4min 20s, sys: 1min 31s, total: 5min 51s
Wall time: 5min 56s


TrainOutput(global_step=500, training_loss=0.9437066955566407, metrics={'train_runtime': 356.1093, 'train_samples_per_second': 22.465, 'train_steps_per_second': 1.404, 'total_flos': 1857401389056000.0, 'train_loss': 0.9437066955566407, 'epoch': 5.0})

In [38]:
hint0 = "18、19日実施の毎日新聞世論調査で、選択的夫婦別姓制度を導入することに賛成かどうかを聞いた。「賛成」は42％で、「反対」は23％。「どちらとも言えない」は34％だった。"
question0 = "選択的夫婦別姓の賛成者はどのくらいいますか？"
## answer = "23%"

gen = generate_sentence(
    f"{start_block}hint\n{hint0}{end_block}\n{start_block}user\n{question0}{end_block}\n{start_block}assistant\n",
    tokenizer,
    trainer.model
).split(f"{start_block}assistant ")[1].replace(end_block, "")

print(gen)

42%


In [35]:
hint0 = "石破茂首相は17日の衆院予算委員会で、小学校給食の無償化について「2026年度以降できる限り早期の制度化を目指したい」と表明した。中学校給食についても「可能な限り速やかに実現したい」と言及。社会保険料負担が生じる「130万円の壁」対策では、25年度中の臨時措置を検討していると明らかにした。"
question0 = "小学校給食の無償化はいつ制度化されますか？"
## answer = "2026年以降"

gen = generate_sentence(
    f"{start_block}hint\n{hint0}{end_block}\n{start_block}user\n{question0}{end_block}\n{start_block}assistant\n",
    tokenizer,
    trainer.model
).split(f"{start_block}assistant ")[1].replace(end_block, "")

print(gen)

2026年度以降
